# Creating a data set for model training from 2021 season data

In [15]:
#Imports
import statsapi
import datetime 
import pandas as pd

In [2]:
#Getting start date and end date of a season
def season_start_end(year):
    season_info = statsapi.get("seasons", {"sportId": 1, "season": year})
    season_start = season_info['seasons'][0]['regularSeasonStartDate']
    season_end = season_info['seasons'][0]['regularSeasonEndDate']
    return [season_start, season_end]

In [3]:
start_end_2021 = season_start_end(2021)
season_start = start_end_2021[0]
season_end = start_end_2021[1]

In [4]:
#Adding 35 days to the season start, when I want to start collecting data
season_start_offset = (datetime.datetime.strptime(season_start, "%Y-%m-%d") + datetime.timedelta(days=35)).strftime('%Y-%m-%d')

In [5]:
#Geting all season matchup outcomes
def season_match_outcomes(season_start, season_end):
    dict_list = []
    for i in statsapi.schedule(start_date=season_start, end_date=season_end):
        match_dict = {}
        try:
            match_dict['game_id'] = i['game_id']
            match_dict['win'] = i['winning_team']
            match_dict['loss'] = i['losing_team']
            dict_list.append(match_dict)
        except: pass
    return dict_list

In [6]:
#Setting variable to the output for season match outcomes
all_matches = season_match_outcomes(season_start_offset, season_end)

In [7]:
#Creating a list of all game ids for period in 2021 season
game_ids = [i['game_id'] for i in all_matches]

In [ ]:
game_info = statsapi.boxscore_data(632520, timecode=None)
game_info

In [ ]:
#Takes a long time to run (16min) - There was a better way I am sure, a df would have been better, but using this for now
#Commenting out to not run again
#dict_list = []
#for i in game_ids:
#    game_info = statsapi.boxscore_data(i, timecode=None)
#    roster_dict = {}
#    roster_dict['game_id'] = int(i)
#    roster_dict['home_roster'] = game_info['home']['battingOrder'] + game_info['home']['bullpen']
#    roster_dict['away_roster'] = game_info['away']['battingOrder'] + game_info['away']['bullpen']
#    dict_list.append(roster_dict)
#dict_list

In [53]:
#Creating json from above dict_list
#Commenting out to not run again
#import json
#with open('roster_dict', 'w') as fout:
#    json.dump(dict_list, fout)

In [ ]:
#Create function to get player stats
#Inputs are the id, start date and end date
def player_stats(id,start_date, end_date, season):
    player_info = statsapi.get("people", {"personIds": id, "hydrate": f"stats(group=[hitting,fielding,pitching],type=[byDateRange],\
    startDate={start_date},endDate={end_date},season={season})"})
    stat_dict = {}
    try:
        player_hitting_stats = player_info['people'][0]['stats'][0]['splits'][0]['stat']
        stat_dict['hitting'] = player_hitting_stats
    except: pass
    try: 
        player_fielding_stats = player_info['people'][0]['stats'][1]['splits'][0]['stat']
        stat_dict['fielding'] = player_fielding_stats
    except: pass
    try:
        player_pitching_stats = player_info['people'][0]['stats'][2]['splits'][0]['stat']
        stat_dict['pitching'] = player_pitching_stats
    except:pass
    return stat_dict

In [ ]:
#Testing function for 2020
player_stats(592450,'2020-07-07','2020-10-22', 2020)